In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.exceptions import UndefinedMetricWarning
import os
import time
import numpy as np
import warnings
# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

from models import LogisticRegression, MLP, LSTMModel, BidirectionalLSTMModel

# Load data
data_dir = 'sequences'
data = torch.load(os.path.join(data_dir, 'data_v4_6416045.pt'))
X = data['X']
y = data['y']

# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Create DataLoaders
batch_size = 16
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a directory to save the best models
models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

def train(model, train_loader, val_loader, epochs=10, patience=5, model_class=None, description=None):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1).float())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Validation
        val_loss = validate(model, val_loader, criterion)
        
        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}')
        
        if description:
            model_path = os.path.join(models_dir, f'{description}_{model_class.__name__.lower()}.pth')
        else:
            model_path = os.path.join(models_dir, f'{model_class.__name__.lower()}.pth')

        # Early stopping and model saving
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print('Early stopping!')
                break  # Exit the loop immediately

    # Load the best model before returning
    model.load_state_dict(torch.load(model_path))
    return model

def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1).float())
            total_loss += loss.item()
    return total_loss / len(val_loader)


def evaluate(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            preds = torch.sigmoid(outputs) > 0.5
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    print('Accuracy:', accuracy)
    # print('Classification Report:')
    # print(classification_report(all_labels, all_preds))
    # print('Confusion Matrix:')
    # print(confusion_matrix(all_labels, all_preds))
    return accuracy

# Get input shape from data
input_shape = X.shape[1:]  # Assuming X is your data tensor

# Update the models dictionary
models = {
    'Logistic Regression': LogisticRegression(input_dim=np.prod(input_shape)),
    'MLP': MLP(input_dim=np.prod(input_shape)),
    'LSTM': LSTMModel(input_size=input_shape[-1], hidden_size=128, num_layers=1),
    'Bidirectional LSTM': BidirectionalLSTMModel(input_size=input_shape[-1], hidden_size=128, num_layers=1),
}

train_description = "v4.2"

# Train and evaluate each model
results = {}
for name, model in models.items():
    print(f"\nTraining {name}")
    start_time = time.time()
    train(model, train_loader, val_loader, epochs=1500, patience=10, model_class=type(model), description=train_description)
    train_time = time.time() - start_time

    print(f"\nEvaluating {name}")
    accuracy = evaluate(model, test_loader)
    
    results[name] = {'accuracy': accuracy, 'train_time': train_time}

# Print summary of results
print("\nSummary of Results:")
for name, result in results.items():
    print(f"{name}: Accuracy = {result['accuracy']:.4f}, Training Time = {result['train_time']:.2f} seconds")


Training Logistic Regression
Epoch 100, Train Loss: 0.2624, Val Loss: 0.2518
Epoch 200, Train Loss: 0.2530, Val Loss: 0.2479
Early stopping!

Evaluating Logistic Regression
Accuracy: 0.8785942492012779

Training MLP
Early stopping!

Evaluating MLP
Accuracy: 0.8977635782747604

Training LSTM
Early stopping!

Evaluating LSTM
Accuracy: 0.9073482428115016

Training Bidirectional LSTM
Early stopping!

Evaluating Bidirectional LSTM
Accuracy: 0.8434504792332268

Summary of Results:
Logistic Regression: Accuracy = 0.8786, Training Time = 9.27 seconds
MLP: Accuracy = 0.8978, Training Time = 4.38 seconds
LSTM: Accuracy = 0.9073, Training Time = 21.49 seconds
Bidirectional LSTM: Accuracy = 0.8435, Training Time = 18.67 seconds


In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score

def evaluate(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            preds = torch.sigmoid(outputs) > 0.5
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    conf_matrix = confusion_matrix(all_labels, all_preds)
    class_report = classification_report(all_labels, all_preds, output_dict=True)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': conf_matrix,
        'classification_report': class_report
    }

# Load and evaluate each model
results = {}
for name in models.keys():
    
    
    # Load the trained model
    model_path = os.path.join(models_dir, f'{train_description}_{name.replace(" ", "_").lower()}.pth')
    print(f"\nEvaluating {model_path}")
    model = torch.load(model_path)
    model.to(device)
    
    # Evaluate the model
    metrics = evaluate(model, test_loader)
    
    results[model_path] = metrics

# Print detailed results
print("\nDetailed Results:")
for name, metrics in results.items():
    print(f"\n{name}:")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print("Confusion Matrix:")
    print(metrics['confusion_matrix'])
    print("Classification Report:")
    # print(classification_report(metrics['classification_report']))

# Print summary of results
print("\nSummary of Results:")
for name, metrics in results.items():
    print(f"{name}: Accuracy = {metrics['accuracy']:.4f}, F1 Score = {metrics['f1']:.4f}")


Evaluating models\v4_logistic_regression.pth

Evaluating models\v4_mlp.pth

Evaluating models\v4_lstm.pth

Evaluating models\v4_bidirectional_lstm.pth


FileNotFoundError: [Errno 2] No such file or directory: 'models\\v4_bidirectional_lstm.pth'